<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Watermark_Processing_in_Streaming_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Watermark Processing in Streaming Systems

Scenario:
Imagine you are processing a real-time stream of sensor data from a weather monitoring system. Each event contains a timestamp indicating when the measurement was taken. Due to network latency or device issues, some events may arrive late, out of order, or significantly delayed.

Your goal is to calculate the average temperature for every 5-minute window while ensuring that late events are included without delaying the computation indefinitely.

Solution: Using Watermarks

Watermark Definition:
A watermark in a streaming system is a marker indicating that all events with timestamps earlier than the watermark's value have likely been received.
For example, if the watermark is set to T - 5 seconds, it signals that the system assumes all events up to 5 seconds before time T have been received.

Handling Late Events:
When an event arrives after its corresponding window has already been processed, it is considered late. Watermarks allow us to balance between waiting for late events and ensuring timely computations.



Event(timestamp=12:00:01, temperature=22.5)
Event(timestamp=12:00:03, temperature=23.0)
Event(timestamp=12:00:02, temperature=22.8)  # Out of order


Windowing Logic: Divide data into 5-minute windows (e.g., 12:00:00–12:05:00).

Watermark Generation:
Watermark is set to CurrentEventTime - 2 seconds. This means the system assumes no more events earlier than 2 seconds before the current processing time will arrive.

Late Event Handling:
If an event arrives late but before the watermark, it is included in the computation. If it arrives after the watermark, it may be discarded or handled in a special "late events" pipeline.

In [ ]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.time_characteristic import TimeCharacteristic
from pyflink.datastream.watermark_strategy import WatermarkStrategy
from pyflink.datastream.window import TimeWindow

from datetime import timedelta


def process_temperature_stream():
    # Set up the environment
    env = StreamExecutionEnvironment.get_execution_environment()
    env.set_stream_time_characteristic(TimeCharacteristic.EventTime)

    # Define the data source
    events = env.from_collection(
        [
            {"timestamp": 1672531201000, "temperature": 22.5},  # 12:00:01
            {"timestamp": 1672531203000, "temperature": 23.0},  # 12:00:03
            {"timestamp": 1672531202000, "temperature": 22.8},  # 12:00:02
        ],
        type_info={"timestamp": int, "temperature": float}
    )

    # Assign timestamps and watermarks
    watermark_strategy = (
        WatermarkStrategy.for_bounded_out_of_orderness(timedelta(seconds=2))
        .with_timestamp_assigner(lambda event, _: event["timestamp"])
    )
    events = events.assign_timestamps_and_watermarks(watermark_strategy)

    # Apply a window function to calculate the average temperature
    result = (
        events
        .key_by(lambda event: "temperature")  # Single key for simplicity
        .window(TimeWindow.of(timedelta(minutes=5)))
        .reduce(
            lambda event1, event2: {
                "timestamp": event1["timestamp"],
                "temperature": (event1["temperature"] + event2["temperature"]) / 2,
            }
        )
    )

    # Print the results
    result.print()

    # Execute the environment
    env.execute("Temperature Streaming with Watermarks")


if __name__ == "__main__":
    process_temperature_stream()


Key Concepts
WatermarkStrategy:

for_bounded_out_of_orderness: Configures the watermark to allow for out-of-order events with a delay of 2 seconds.
with_timestamp_assigner: Extracts the timestamp from each event for ordering.
Windowing:

Data is grouped into 5-minute time windows for aggregation.
Late Events Handling:

Events arriving after the watermark are discarded or sent to a side-output for special processing.
Reduce Function:

Aggregates temperatures within each window to calculate the average.


How Watermarks Handle Late Events
On-Time Events: Processed normally within their corresponding window.
Late Events (Within Watermark Delay): Included in the current window computation.
Late Events (Beyond Watermark): Either discarded or sent to a separate stream for analysis.

# radiation dose sensor data

Adapting the Workflow
Here’s how the watermark processing concept can be tailored for radiation dose data:

Streaming Data Source:

Radiation dose readings are streamed in real-time, each with a timestamp and dose value.
Example: {"timestamp": 1672531201000, "dose": 0.05} (Timestamp in milliseconds since epoch).
Windowing Logic:

Define a time-based window (e.g., 5-minute rolling windows) to aggregate or compute statistics like average dose or max dose.
Watermark Strategy:

Use watermarks to handle out-of-order or late-arriving data. For example, allow a bounded delay of 2 seconds, meaning data arriving within 2 seconds of the window end is still included.
Processing Goals:

Timely Results: Ensure results are computed without waiting indefinitely for late events.
Late Event Handling: Include late events within the watermark delay, but log or analyze events arriving too late.


In [ ]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.time_characteristic import TimeCharacteristic
from pyflink.datastream.watermark_strategy import WatermarkStrategy
from pyflink.datastream.window import TimeWindow
from datetime import timedelta


def process_radiation_stream():
    # Set up the Flink environment
    env = StreamExecutionEnvironment.get_execution_environment()
    env.set_stream_time_characteristic(TimeCharacteristic.EventTime)

    # Simulated radiation dose sensor data
    radiation_data = env.from_collection(
        [
            {"timestamp": 1672531201000, "dose": 0.05},  # 12:00:01
            {"timestamp": 1672531203000, "dose": 0.06},  # 12:00:03
            {"timestamp": 1672531202000, "dose": 0.07},  # 12:00:02
            {"timestamp": 1672531230000, "dose": 0.08},  # 12:00:30
        ],
        type_info={"timestamp": int, "dose": float}
    )

    # Assign watermarks to handle late events
    watermark_strategy = (
        WatermarkStrategy.for_bounded_out_of_orderness(timedelta(seconds=2))
        .with_timestamp_assigner(lambda event, _: event["timestamp"])
    )
    radiation_data = radiation_data.assign_timestamps_and_watermarks(watermark_strategy)

    # Apply windowed aggregation to compute average radiation dose
    average_dose = (
        radiation_data
        .key_by(lambda _: "dose")  # Single key for simplicity
        .window(TimeWindow.of(timedelta(minutes=5)))
        .reduce(
            lambda event1, event2: {
                "timestamp": event1["timestamp"],
                "dose": (event1["dose"] + event2["dose"]) / 2
            }
        )
    )

    # Print the results
    average_dose.print()

    # Execute the Flink pipeline
    env.execute("Radiation Dose Streaming with Watermarks")


if __name__ == "__main__":
    process_radiation_stream()


Enhancements for Radiation-Specific Applications
To make the workflow more effective for radiation dose data, consider the following:

Threshold Alerts:

Add a filter to trigger alerts if the average dose exceeds a safety threshold.
Example:
python
Copy code
if avg_dose > threshold:
    send_alert(avg_dose)
Late Event Analysis:

Store late events in a separate stream for post-hoc analysis.
This helps understand patterns in delayed sensor data.
Data Visualization:

Integrate the pipeline with visualization tools like Grafana or Matplotlib for monitoring.
Anomaly Detection:

Use machine learning models to detect anomalies in the radiation dose data stream.
Scalability:

If handling data from multiple sensors, extend the pipeline to include sensor-specific keys and aggregation.